In [ ]:
include("../analysis/mESC/load_analysis.jl")

In [ ]:
# fix the replication point to be in the middle of the S phase
θᵣ = θ_G1_S + (θ_S_G2M - θ_G1_S)/2

In [113]:
cv3 = colorant"#57C6FF"
cv2 = colorant"#9881FD"
cv1 = c1
cb = colorant"#40b2dd";

# Fig. 3A: counts versus $\theta$

In [ ]:
function linear_regression(x, y)
    X = [ones(eltype(x), length(x)) x]
    beta = X \ y
    y_pred = X*beta
    beta, y_pred
end

In [ ]:
f = Figure(size = (size_pt[1]*0.9, size_pt[2]*1.4), figure_padding=1)
ax = Axis(f[1, 1], xlabel="Cell age θ", ylabel="Counts", 
          xticks=0:0.2:1)

x = theta
xth = sort(unique(x))
θs = sort(unique(theta))

ncells = 1000

ind = findfirst(gene_names .== "Tpt1")
y = counts_spliced[ind]
mean_y = [mean(y[findall(th .== theta)]) for th in θs]
rinds = shuffle(1:length(y))[1:ncells]
scatter!(ax, x[rinds], y[rinds], color=(cv1, 0.45), markersize=3)
lines!(ax, xth, mean_y, color=(:black, 0.45), linewidth=0.8)
println(cor(x, y))
println(cor(x, y)^2)

ind = findfirst(gene_names .== "Atp5b")
y = counts_spliced[ind]
mean_y = [mean(y[findall(th .== theta)]) for th in θs]
rinds = shuffle(1:length(y))[1:ncells]
scatter!(ax, x[rinds], y[rinds], color=(cv2, 0.35), markersize=3)
lines!(ax, xth, mean_y, color=(:black, 0.45), linewidth=0.8)
println(cor(x, y))
println(cor(x, y)^2)

ind = findfirst(gene_names .== "Pcbp1")
y = counts_spliced[ind] 
mean_y = [mean(y[findall(th .== theta)]) for th in θs]
rinds = shuffle(1:length(y))[1:ncells]
scatter!(ax, x[rinds], y[rinds], color=(cv3, 0.5), markersize=3)
lines!(ax, xth, mean_y, color=(:black, 0.45), linewidth=0.8)
beta, rsq = linear_regression(x, y)
println(cor(x, y))
println(cor(x, y)^2)

#vlines!(ax, θ_G1_S, linewidth=0.8, color=(:black, 0.3))
#vlines!(ax, θ_S_G2M, linewidth=0.8, color=(:black, 0.3))

ylims!(ax, -2, 190)
xlims!(-0.02, 1.01)

f

# Fig. 3B: correlation between mean counts and $\theta$

In [ ]:
function plot_counts(f::GridPosition, ind::Int)
    
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    counts_S = [xS[ind][_inds] for _inds in inds_θs_S]
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]
    
    yG1 = mean.(counts_G1)
    yS = mean.(counts_S)
    yG2M = mean.(counts_G2M)
    #, yticks=(6:4:14)
    ax = Axis(f, xlabel="Cell age θ", ylabel="Mean counts", xticks=(0:0.2:1))
    #ax = Axis(f[1, 1], xlabel="Cell age θ", ylabel="Mean counts", xticks=(0:0.2:1), yticklabelsvisible=false)
    vlines!(ax, θ_G1_S, linewidth=0.9, color=(:black, 0.2))
    vlines!(ax, θ_S_G2M, linewidth=0.9, color=(:black, 0.2))
    vlines!(ax, θᵣ, linewidth=0.9, color=(cv2, 0.6))

    x = θs
    y = vcat(yG1, yS, yG2M)
    lines!(ax, x, y, color=(cb, 0.3), linewidth=0.3)
    #scatter!(ax, x, y, color=(cb, 0.7), markersize=2.0, strokecolor=(c1, 0.85), strokewidth=0.1)
    scatter!(ax, x, y, color=(cb, 0.7), markersize=3.0, strokecolor=(c1, 0.85), strokewidth=0.2)
    
    #ylims!(ax, low=4, high=15)
    xlims!(ax, low=-0.02, high=1.01)
    ax
end

In [122]:
f = Figure(size = (size_pt[1]*2.05, size_pt[2]*0.65), figure_padding = 1)
ga = GridLayout(f[1,1])

total_xG1 = vec(sum(hcat(xG1...), dims=2))
total_xS = vec(sum(hcat(xS...), dims=2))
total_xG2M = vec(sum(hcat(xG2M...), dims=2))

counts_G1 = [total_xG1[_inds] for _inds in inds_θs_G1]
counts_S = [total_xS[_inds] for _inds in inds_θs_S]
counts_G2M = [total_xG2M[_inds] for _inds in inds_θs_G2M]

yG1 = mean.(counts_G1)
yS = mean.(counts_S)
yG2M = mean.(counts_G2M)
min_ind = findmin(vcat(yG1, yS, yG2M))[2]

ax1 = Axis(ga[1, 1], xlabel="", ylabel="Mean counts", 
           xticks=(0:0.2:1), #xticklabelsvisible=false,
           yticks=([2e4, 3e4, 4e4], ["2×10⁴", "3×10⁴", "4×10⁴"]))
vlines!(ax1, θ_G1_S, linewidth=0.9, color=(:black, 0.2))
vlines!(ax1, θ_S_G2M, linewidth=0.9, color=(:black, 0.2))
vlines!(ax1, θᵣ, linewidth=0.9, color=(cv2, 0.6))

x = θs
y = vcat(yG1, yS, yG2M)
lines!(ax1, x, y, color=(cb, 0.3), linewidth=0.3)
#scatter!(ax1, x, y, color=(cb, 0.7), markersize=2.0, strokecolor=(c1, 0.85), strokewidth=0.1)
scatter!(ax1, x, y, color=(cb, 0.7), markersize=3.0, strokecolor=(c1, 0.85), strokewidth=0.2)

xlims!(ax1, low=-0.02, high=1.01)
#ylims!(ax1, low=0.8e4, high=2.8e4)

ind = findfirst(gene_names .== "Sparc")
ax2 = plot_counts(ga[1, 2], ind)
ylims!(ax2, low=3.3, high=11.9)
ax2.ylabel = ""

colgap!(ga, 6)

In [ ]:
f

In [124]:
θs_S_1 = θs_S[θs_S .<= θᵣ]
θs_S_2 = θs_S[θs_S .> θᵣ];

In [125]:
inds_θs_S_1 = [findall(th .== thetaS) for th in θs_S_1]
inds_θs_S_2 = [findall(th .== thetaS) for th in θs_S_2];

In [ ]:
function get_corr_G1(ind::Int)
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    yG1 = mean.(counts_G1)
    cor(θs_G1, yG1)
end

function get_corr_S_1(ind::Int)
    counts_S_1 = [xS[ind][_inds] for _inds in inds_θs_S_1]
    yS_1 = mean.(counts_S_1)
    cor(θs_S_1, yS_1)
end

function get_corr_S_2(ind::Int)
    counts_S_2 = [xS[ind][_inds] for _inds in inds_θs_S_2]
    yS_2 = mean.(counts_S_2)
    cor(θs_S_2, yS_2)
end

function get_corr_G2M(ind::Int)
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]
    yG2M = mean.(counts_G2M)
    cor(θs_G2M, yG2M)
end

In [ ]:
y1 = get_corr_G1.(1:ngenes); x1 = fill("G1", length(y1))
y2 = get_corr_S_1.(1:ngenes); x2 = fill("S₁", length(y2))
y3 = get_corr_S_2.(1:ngenes); x3 = fill("S₂", length(y3))
y4 = get_corr_G2M.(1:ngenes); x4 = fill("G2/M", length(y4));

f = Figure(size = (size_pt[1]*0.85, size_pt[2]*0.95), figure_padding = 1)
ax = Axis(f[1, 1], xlabel="Correlation coefficient r", ylabel="", yticks=(1:4, ["G1", "Early S", "Late S", "G2/M"]))
ax = rainclouds!(ax, vcat(x1, x2, x3, x4), vcat(y1, y2, y3, y4), gap=-1.2, 
                orientation = :horizontal,
                color = vcat(fill(cx1, length(y1)), fill(cx2, length(y2)), fill(cx2, length(y2)), fill(cx3, length(y3))),
                cloud_width=0.6, show_median=false, violin_limits=(-Inf, Inf),
                boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
                jitter_width=0.15, markersize=1.3, side_nudge=0.095)
vlines!(0, color=(:black, 0.5), linestyle=:dash)
xlims!(-1, 1)
f

# Fig. 3B: nonlinear regression

In [128]:
using LsqFit
nonlinear_fit(θ, p) = p[1] * exp.(p[2] * θ);

function get_params_lsq_G1_nonlinear(ind::Int)
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    yG1 = mean.(counts_G1)
    ps_G1 = curve_fit(nonlinear_fit, θs_G1, yG1, [1.0, 1.0]).param
    ps_G1
end

function get_params_lsq_S_1_nonlinear(ind::Int)
    counts_S_1 = [xS[ind][_inds] for _inds in inds_θs_S_1]
    yS_1 = mean.(counts_S_1)
    ps_S_1 = curve_fit(nonlinear_fit, θs_S_1, yS_1, [1.0, 1.0]).param
    ps_S_1
end

function get_params_lsq_S_2_nonlinear(ind::Int)
    counts_S_2 = [xS[ind][_inds] for _inds in inds_θs_S_2]
    yS_2 = mean.(counts_S_2)
    ps_S_2 = curve_fit(nonlinear_fit, θs_S_2, yS_2, [1.0, 1.0]).param
    ps_S_2
end

function get_params_lsq_G2M_nonlinear(ind::Int)
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]
    yG2M = mean.(counts_G2M)
    ps_G2M = curve_fit(nonlinear_fit, θs_G2M, yG2M, [1.0, 1.0]).param
    ps_G2M
end;

In [129]:
ps_G1_nonlinear = get_params_lsq_G1_nonlinear.(1:ngenes)
ps_S_1_nonlinear = get_params_lsq_S_1_nonlinear.(1:ngenes)
ps_S_2_nonlinear = get_params_lsq_S_2_nonlinear.(1:ngenes)
ps_G2M_nonlinear = get_params_lsq_G2M_nonlinear.(1:ngenes)

k_G1 = [ps[1] for ps in ps_G1_nonlinear]
k_S_1 = [ps[1] for ps in ps_S_1_nonlinear]
k_S_2 = [ps[1] for ps in ps_S_2_nonlinear]
k_G2M = [ps[1] for ps in ps_G2M_nonlinear]

beta_G1 = [ps[2] for ps in ps_G1_nonlinear]
beta_S_1 = [ps[2] for ps in ps_S_1_nonlinear]
beta_S_2 = [ps[2] for ps in ps_S_2_nonlinear]
beta_G2M = [ps[2] for ps in ps_G2M_nonlinear];

In [ ]:
f = Figure(size = (size_pt[1]*2.45, size_pt[2]*0.95), figure_padding = 1)
ga = GridLayout(f[1,1])

y1 = get_corr_G1.(1:ngenes); x1 = fill("G1", length(y1))
y2 = get_corr_S_1.(1:ngenes); x2 = fill("S₁", length(y2))
y3 = get_corr_S_2.(1:ngenes); x3 = fill("S₂", length(y3))
y4 = get_corr_G2M.(1:ngenes); x4 = fill("G2/M", length(y4));

ax1 = Axis(ga[1, 1], xlabel="Correlation coefficient r", ylabel="", yticks=(1:4, ["G1", "Early S", "Late S", "G2/M"]))
rainclouds!(ax1, vcat(x1, x2, x3, x4), vcat(y1, y2, y3, y4), gap=-1.0, 
            orientation = :horizontal,
            color = vcat(fill(cx1, length(y1)), fill(cx2, length(y2)), fill(cx2, length(y2)), fill(cx3, length(y3))),
            cloud_width=0.6, show_median=false, violin_limits=(-Inf, Inf),
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.15, markersize=1.3, side_nudge=0.1)
vlines!(ax1, 0, color=(:black, 0.4), linestyle=:dash)
xlims!(ax1, -1, 1)

y1 = k_G1; x1 = fill("G1", length(y1))
y2 = k_S_1; x2 = fill("S₁", length(y2))
y3 = k_S_2; x3 = fill("S₂", length(y3))
y4 = k_G2M; x4 = fill("G2/M", length(y4));

ax2 = Axis(ga[1, 2], xlabel="Proportionality constant k", ylabel="", yticklabelsvisible=false)
rainclouds!(ax2, vcat(x1, x2, x3, x4), vcat(y1, y2, y3, y4), gap=-1.0, 
            orientation = :horizontal,
            color = vcat(fill(cx1, length(y1)), fill(cx2, length(y2)), fill(cx2, length(y2)), fill(cx3, length(y3))),
            cloud_width=0.6, show_median=false, violin_limits=(-Inf, Inf),
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.15, markersize=1.3, side_nudge=0.1)
vlines!(ax2, 0, color=(:black, 0.4), linestyle=:dash)
xlims!(ax2, -2, 32)

y1 = beta_G1; x1 = fill("G1", length(y1))
y2 = beta_S_1; x2 = fill("S₁", length(y2))
y3 = beta_S_2; x3 = fill("S₂", length(y3))
y4 = beta_G2M; x4 = fill("G2/M", length(y4));

ax3 = Axis(ga[1, 3], xlabel="Scaling exponent β", ylabel="", yticklabelsvisible=false)
rainclouds!(ax3, vcat(x1, x2, x3, x4), vcat(y1, y2, y3, y4), gap=-1.0, 
            orientation = :horizontal,
            color = vcat(fill(cx1, length(y1)), fill(cx2, length(y2)), fill(cx2, length(y2)), fill(cx3, length(y3))),
            cloud_width=0.6, show_median=false, violin_limits=(-Inf, Inf),
            boxplot_width=0.1, boxplot_nudge=0.0, strokewidth = 0.7, whiskerwidth=0,
            jitter_width=0.15, markersize=1.3, side_nudge=0.1)
vlines!(ax3, 0, color=(:black, 0.4), linestyle=:dash)
xlims!(ax3, -4.5, 4.5)

colgap!(ga, 8)

f

# Fig. 3C: example distributions

In [ ]:
include("../analysis/mESC/filter_prior.jl")

srcpath = normpath(srcpath*"../age-dependent/")
fitpath = datapath*"fits_age-dependent/"
include(srcpath*"dists.jl")  
include(srcpath*"mle.jl")

fits_main = load(fitpath*"fits_main.jld2", "fits_main")
include("../analysis/mESC/filter_post.jl");

In [ ]:
function stairpts(s)
    pts = s.plots[1].converted[1][]
    [p[1] for p in pts], [p[2] for p in pts]
end

function plot_fit(f::GridPosition, xdata::AbstractArray, d::Distribution; kwargs...)
    ax = Axis(f, xlabel="Counts", ylabel="Probability", kwargs...)
    xs, ww = fit_hist(xdata)
    ww = ww ./ ww.sum
    
    ylims!(0, nothing)
    xlims!(-0.8, xs[end]+0.8)
    
    s = stairs!(ax, vcat(-1, xs, xs[end]+1), vcat(0, ww, 0), 
               step=:center, color=(cx1, 1), linewidth=0)

    xs′, ys′ = stairpts(s)
    band!(ax, xs′, 0*ys′, ys′, color=(colorant"#d0d0d1", 1))
    vlines!(ax, mean(xdata), linewidth=1, color=(cx1, 0.8))
    lines!(ax, xs, pdf.(d, xs), color=(c3, 1), linewidth=1)
    ax
end

function plot_fit(xdata::AbstractArray, d::Distribution; kwargs...)
    f = Figure(size = (size_pt[1]*0.65, size_pt[2]*0.7), figure_padding = 3)
    plot_fit(f[1,1], xdata, d, kwargs...)
    f
end

In [ ]:
ind = findfirst(gene_names .== "Bex2")
th_ind = 8
th = θs_G1[th_ind]
println("θ = $th")
data = xG1[ind][inds_θs_G1[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
@show mean(data)
@show mean(d)
plot_fit(data, d)

In [ ]:
ind = findfirst(gene_names .== "Cox6a1")
th_ind = 22
th = θs_S[th_ind]
println("θ = $th")
data = xS[ind][inds_θs_S[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
@show mean(data)
@show mean(d)
plot_fit(data, d)


In [ ]:
ind = findfirst(gene_names .== "Cox6a1")
th_ind = 14
th = θs_G2M[th_ind]
println("θ = $th")
data = xG2M[ind][inds_θs_G2M[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
@show mean(data)
@show mean(d)
plot_fit(data, d)

In [ ]:
ind = findfirst(gene_names .== "Ckb")
th_ind = 13
th = θs_G1[th_ind]
println("θ = $th")
data = xG1[ind][inds_θs_G1[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
@show mean(data)
@show mean(d)
@show var(data)
@show var(d)
plot_fit(data, d)

In [ ]:
f = Figure(size = (size_pt[1]*2.4, size_pt[2]*0.6), figure_padding = 1)
ga = f[1,1] = GridLayout()

ind = findfirst(gene_names .== "Bex2")
th_ind = 8
th = θs_G1[th_ind]
#θs_G1[8]
data = xG1[ind][inds_θs_G1[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
plot_fit(data, d)
ax1 = plot_fit(ga[1, 1], data, d)
ylims!(ax1, 0, nothing)
xlims!(ax1, -0.02, nothing)

ind = findfirst(gene_names .== "Cox6a1")
th_ind = 22
th = θs_S[th_ind]
#θs_S[22]
data = xS[ind][inds_θs_S[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
ax2 = plot_fit(ga[1, 2], data, d)
ax2.ylabel = ""
ax2.xlabel = ""
ylims!(ax2, 0, nothing)
xlims!(ax2, -0.02, nothing)

ind = findfirst(gene_names .== "Cox6a1")
th_ind = 14
th = θs_G2M[th_ind]
#θs_G2M[14]
data = xG2M[ind][inds_θs_G2M[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
ax3 = plot_fit(ga[1, 3], data, d)
ax3.ylabel = ""
ax3.xlabel = ""
ylims!(ax3, 0, nothing)
xlims!(ax3, -0.02, nothing)

ind = findfirst(gene_names .== "Ckb")
th_ind = 13
th = θs_G1[th_ind]
#θs_G1[13]
data = xG1[ind][inds_θs_G1[th_ind]]
d = NegativeBinomial(fits_main[ind], th, T_cycle, decay_rates[ind], θᵣ, θ_G1_S, θ_S_G2M)
ax4 = plot_fit(ga[1, 4], data, d)
ax4.ylabel = ""
ax4.xlabel = ""
ylims!(ax4, 0, nothing)
xlims!(ax4, -0.02, nothing)

colgap!(ga, 10)

f

# Fig. 3C: example means/variances

In [ ]:
function plot_mean_fit(f::GridPosition, ind::Int)
    m = fits_main[ind]
    
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    counts_S = [xS[ind][_inds] for _inds in inds_θs_S]
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]

    yG1 = mean.(counts_G1)
    yS = mean.(counts_S)
    yG2M = mean.(counts_G2M)

    ax = Axis(f, xlabel="", ylabel="Mean", xticks=(0:0.2:1))
    vlines!(ax, θ_G1_S, linewidth=0.8, color=(:black, 0.2))
    vlines!(ax, θ_S_G2M, linewidth=0.8, color=(:black, 0.2))
    vlines!(ax, θᵣ, linewidth=0.8, color=(cv2, 0.6))

    y = vcat(yG1, yS, yG2M)
    lines!(ax, θs, y, color=(cb, 0.3), linewidth=0.3)
    scatter!(ax, θs, y, color=(cb, 0.7), markersize=3.0, strokecolor=(c1, 0.85), strokewidth=0.2)
    lines!(ax, θs, mean.(Ref(m), θs, Ref(T_cycle), Ref(decay_rates[ind]), Ref(θᵣ), Ref(θ_G1_S), Ref(θ_S_G2M)), color=(c3, 0.7), linewidth=1.8)

    xlims!(ax, low=-0.02, high=1.01)

    ax 
end

In [ ]:
function plot_var_fit(f::GridPosition, ind::Int)
    m = fits_main[ind]
    
    counts_G1 = [xG1[ind][_inds] for _inds in inds_θs_G1]
    counts_S = [xS[ind][_inds] for _inds in inds_θs_S]
    counts_G2M = [xG2M[ind][_inds] for _inds in inds_θs_G2M]

    yG1 = var.(counts_G1)
    yS = var.(counts_S)
    yG2M = var.(counts_G2M)
 
    ax = Axis(f, xlabel="Cell age θ", ylabel="Variance", xticks=(0:0.2:1))
    vlines!(ax, θ_G1_S, linewidth=0.8, color=(:black, 0.2))
    vlines!(ax, θ_S_G2M, linewidth=0.8, color=(:black, 0.2))
    vlines!(ax, θᵣ, linewidth=0.8, color=(cv2, 0.6))
        
    y = vcat(yG1, yS, yG2M)
    lines!(ax, θs, y, color=(cb, 0.3), linewidth=0.3)
    scatter!(ax, θs, y, color=(cb, 0.7), markersize=3.0, strokecolor=(c1, 0.85), strokewidth=0.2)
    lines!(ax, θs, var.(Ref(m), θs, Ref(T_cycle), Ref(decay_rates[ind]), Ref(θᵣ), Ref(θ_G1_S), Ref(θ_S_G2M)), color=(c3, 0.7), linewidth=1.8)
    
    xlims!(ax, low=-0.02, high=1.01)
    ax 
end

In [ ]:
f = Figure(size = (size_pt[1]*3.0, size_pt[2]*1.4), figure_padding = 1, fontsize=7)
ga = f[1,1] = GridLayout()

ind = findfirst(gene_names .== "Bex2")
ax11 = plot_mean_fit(ga[1,1], ind)
ax11.title = gene_names[ind]
ax11.titlefont = "Arial"
ax11.xlabel = ""
ax11.xticksvisible = false
ax11.xticklabelsvisible = false

ax21 = plot_var_fit(ga[2,1], ind)
ax21.xlabel = ""
ax21.yticks = 3:3:12

ind = findfirst(gene_names .== "Cox6a1")
ax12 = plot_mean_fit(ga[1,2], ind)
ax12.title = gene_names[ind]
ax12.titlefont = "Arial"
ax12.xlabel = ""
ax12.ylabel = ""
ax12.xticksvisible = false
ax12.xticklabelsvisible = false

ax22 = plot_var_fit(ga[2,2], ind)
ax22.ylabel = ""
ax22.yticks = 10:20:50

ind = findfirst(gene_names .== "Ckb")
ax13 = plot_mean_fit(ga[1,3], ind)
ax13.title = gene_names[ind]
ax13.titlefont = "Arial"
ax13.xlabel = ""
ax13.ylabel = ""
ax13.xticksvisible = false
ax13.xticklabelsvisible = false

ax23 = plot_var_fit(ga[2,3], ind)
ax23.xlabel = ""
ax23.ylabel = ""
ax23.yticks = 200:500:2000

colgap!(ga, 10)
rowgap!(ga, 6)

f